# Read outputs of analyze_functionalized for further custom analysis

In [ ]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import sys
sys.path.append('ext')
import plot_util
import accelerometers

In [ ]:
inputdir = pathlib.Path.cwd() / 'outputdata'
# inputdir is previous outputdir
outputdir = pathlib.Path.cwd() / 'outputdata' / 'explore'
outputdir.mkdir(exist_ok=True)

pathlib.Path.cwd()

In [ ]:
combined = pd.read_csv(inputdir / 'Combined.csv', header=[0, 1], index_col=0)
combined.head()

In [ ]:
# Recreate standard combined average plot
combined_bycat_log = combined.map(np.log10).groupby('Category', axis=1).mean()
plot_util.generic_plot(combined_bycat_log.rename(np.log10, axis=0), kind='line', xlabel='Hz', ylabel='W/kg/Hz',
                       output_directory=outputdir, title='Average Power Spectra by Mode',
                       logx=True, logy=True)

From here down may need customization

In [ ]:
# Let's look just at BART and Muni
plot_util.generic_plot(combined_bycat_log.rename(np.log10, axis=0)[['Muni Train', 'BART']], kind='line', xlabel='log10(Hz)', ylabel='log10(W/kg/Hz)',
                       output_directory=outputdir, title='Average Power Spectra by Mode, Selected')

In [ ]:
# What does it look like removing the logy?
plot_util.generic_plot((10**combined_bycat_log).rename(np.log10, axis=0)[['Muni Train', 'BART']], kind='line', xlabel='log10(Hz)', ylabel='W/kg/Hz',
                       output_directory=outputdir, title='Average Power Spectra by Mode, Unlog-y')

In [ ]:
plot_util.generic_plot((10**combined_bycat_log).rename(np.log10, axis=0)[['Muni Train', 'BART']], kind='line', xlabel='log10(Hz)', ylabel='W/kg/Hz',
                       output_directory=outputdir, title='Average Power Spectra by Mode, Unlog-y', xlim=[-0.75, 1.5], ylim=[0, 150])

In [ ]:
# What does the raw BART trip set look like?
idx = pd.IndexSlice
subtrips = combined.loc[idx[:], idx['BART']]
plot_util.generic_plot(subtrips.rename(np.log10, axis=0), kind='line', xlabel='log10(Hz)', ylabel='W/kg/Hz',
                       output_directory=outputdir, title='Trip Power Spectra by Selected Mode')

In [ ]:
# Log them
plot_util.generic_plot(subtrips.rename(np.log10, axis=0).map(np.log10), kind='line', xlabel='log10(Hz)', ylabel='log10(W/kg/Hz)',
                       output_directory=outputdir, title='Logged Trip Power Spectra by Selected Mode')

In [ ]:
combined_bycat_log.columns

In [ ]:
motorized_travel_modes = ['BART', 'Muni Train', 'Muni Bus', 'BensCar']
plot_util.generic_plot(combined_bycat_log[motorized_travel_modes].rename(np.log10, axis=0), kind='line', xlabel='Hz', ylabel='W/kg/Hz',
                       output_directory=outputdir, title='Average Power Spectra for Motorized Travel Modes',
                       logx=True, logy=True)

In [ ]:
# Let's integrate over three regimes
bounds = [10**(-0.5), 10**(0.75)]
bounds

In [ ]:
combined_bycat_log.head()

In [ ]:
pow_seg = pd.DataFrame(index=np.arange(len(bounds)+1), columns=combined_bycat_log.columns)
for i in pow_seg.index:
    if i == 0:
        islice = combined_bycat_log.index <= bounds[i]
    elif i < len(bounds):
        islice = (combined_bycat_log.index > bounds[i-1]) & (combined_bycat_log.index <= bounds[i])
    else:
        islice = combined_bycat_log.index > bounds[i-1]
    segment = combined_bycat_log.loc[islice]
    for col in pow_seg.columns:
        pow_seg.loc[i, col] = np.trapezoid(10**segment[col], x=segment.index)
pow_seg

In [ ]:
labels = {0: 'Bulk Acceleration', 1: 'Oscillation', 2: 'Vibration'}
pow_seg = pow_seg.rename(labels)
plot_util.generic_plot(pow_seg.map(np.log10).T, kind='bar', ylabel='W/kg', unstacked=True, title='Power by Freq. Range & Mode', output_directory=outputdir,
                      logy=True, fontsize=9)

In [ ]:
plot_util.generic_plot(pow_seg[motorized_travel_modes].T, kind='bar', ylabel='W/kg', title='Power by Freq. Range for Motorized Modes', output_directory=outputdir)